In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import SRN2VecModel
import json

In [2]:
network = RoadNetwork()
network.load("../../osm_data/porto")

In [7]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network)

In [ ]:
model.generate_data(n_shortest_paths=100, number_negative=3, window_size=900, save_batch_size=100)

In [ ]:
%lprun -f model.extract_pairs model.generate_train_pairs(network, paths, 900, 7)

In [ ]:
with open("srn2vec-traindata.json", "r") as fp:
    a = json.load(fp)

In [17]:
"""
Build and train model
"""
network = RoadNetwork()
network.load("../../osm_data/porto")
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network, remove_highway_label=True)
model.load_dataset("./srn2vec-traindata.json")

In [18]:
model.train(epochs=5, batch_size=256)

Epoch: 0, Iteration: 0, sample_loss: 0.7244336605072021, Avg. Loss: 0.7244336605072021
Epoch: 0, Iteration: 1000, sample_loss: 0.6133495569229126, Avg. Loss: 0.6258018823651286
Epoch: 0, Iteration: 2000, sample_loss: 0.6045035123825073, Avg. Loss: 0.6102273479990218
Epoch: 0, Iteration: 3000, sample_loss: 0.6053149700164795, Avg. Loss: 0.6046309277678442
Epoch: 0, Iteration: 4000, sample_loss: 0.6026860475540161, Avg. Loss: 0.6016880611990786
Epoch: 0, Iteration: 5000, sample_loss: 0.5851001739501953, Avg. Loss: 0.6000700907167542
Epoch: 0, Iteration: 6000, sample_loss: 0.6103091239929199, Avg. Loss: 0.5989909757576154
Epoch: 0, Iteration: 7000, sample_loss: 0.5944563150405884, Avg. Loss: 0.5981639968448155
Epoch: 0, Iteration: 8000, sample_loss: 0.6560654640197754, Avg. Loss: 0.5974557065781974
Epoch: 0, Iteration: 9000, sample_loss: 0.548535943031311, Avg. Loss: 0.5966067338715578
Epoch: 0, Iteration: 10000, sample_loss: 0.5428558588027954, Avg. Loss: 0.5946249320302983
Epoch: 0, Ite

In [19]:
model.save_model(path="../model_states/srn2vec/")